## An example workflow for D3D

A D3D workflow using local ERAI data.

In [ ]:
#to use the full width of the browser window uncomment the code below and execute the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon
import pyPoseidon.model as pm
import datetime
from pyPoseidon.utils import data,point
from pyPoseidon.utils.vis import video
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pyPoseidon.__version__

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import cartopy.crs as ccrs
from hvplot import pandas,xarray

### Define case

In [ ]:
#define in a dictionary the properties of the model..
dic={'minlon':-35., # lat/lon window
     'maxlon':42.,
     'minlat':25.05,
     'maxlat':76.5,
     'solver':'d3d',
     'tag':'eur01',
     'resolution':0.1, #grid resoltuion 
     'step':60, # step for output of map field in d3d
     'rstep': 'end' ,  # step for restart file
     'ostep': 5, # observation points save step
     'start_date':'2016-1-1 0:0:0',
     'end_date':'2016-1-2 0:0:0',
     'ft1':0, # initial forecasting hour per file
     'ft2':11, # final forecasting hour per file
     'meteo':'erai',
     'dem': 'gebco',
     'dpath' : '/Users/brey/DATA/BATH/GEBCO_2014_2D.nc',
     'epath':'/Users/brey/DELFT3D/SVN/7545/bin/lnx64/', #folder for solver executables
     'ncores': 4 , #number of cores
     'rpath':'/Users/brey/Downloads/EUR/D3D/ERAI/201701/', #location of calc folder
#     'Dt':1.0, # simulation time step
     'config': {'Dt':1.0}, #add any additional relevant setup variable
     'conda_bin' : '/Users/brey/miniconda3/bin', #optional conda bin path
     'conda_env': 'mpi3', # optional conda env for the solver
     'ofilename': '/Users/brey/DATA/OBS/eu.csv',
     'update':[] # optional list of components to be updated
    }

#### Local operational ECMWF files

In [ ]:
start_date = pd.to_datetime(dic['start_date']) 
end_date = pd.to_datetime(dic['end_date']) 

In [ ]:
#keep the years
dr = np.arange(start_date.year,end_date.year+1)
dr

In [ ]:
#Set rstep
dt=end_date-start_date
dt = dt.total_seconds()/60
dic.update({'rstep':int(dt)})

In [ ]:
#creating a sequence of folder from which we read the meteo.

PATH='/Users/brey/DATA/ERAI/'#Path to meteo files

meteo_files = [PATH+'eraInterim_uvp_{:04d}.grib'.format(x) for x in dr]
meteo_files

#### update dictionary

In [ ]:
dic.update({'mpaths':meteo_files})

## Initialize

In [ ]:
#initialize a model
b = pm.model(**dic)

### set it up

In [ ]:
b.set() #set it up 

In [ ]:
b.impl.meteo.impl.uvp

In [ ]:
b.impl.dem.impl.Dataset

In [ ]:
b.impl.dem.impl.Dataset.ival.hvplot('ilon','ilat',width=800, height=600, cmap='viridis')# test coverage

#### Optional adjust the wet area based on a coastline shapefile

In [ ]:
b.impl.dem.impl.adjust('/Users/brey/DATA/COASTLINES/naturalearth/coastline/ne_50m_coastline.shp',**{'nc':50,'ncores':4}) 

In [ ]:
b.impl.dem.impl.Dataset

In [ ]:
b.impl.dem.impl.Dataset.fval.hvplot('ilon','ilat',width=800, height=600, cmap='viridis')# test coverage

## Save to folder for execution 

In [ ]:
#set the run by saving the files
b.output()

In [ ]:
# save model info for further use
b.save()

In [ ]:
# save all matrices for further use
#b.pickle(path=path)

### execute

In [ ]:
#execute
b.run()

## Analysis of output 

In [ ]:
otp = data(**{'solver':'d3d','rpath':dic['rpath']})#,'savenc':True})#,tag='eur025')# optional tag for data selection ,sa_date='2010-2-1 5:0:0',se_date='2010-2-1 22:0:0') # optionally set range for tide gauge data acquisition 

In [ ]:
otp.impl.Dataset

### Visualize

In [ ]:
otp.impl.Dataset.hvplot.quadmesh(x='XZ', y='YZ', z='S1', groupby='time', width=800, height=600,cmap='viridis', dynamic=True, rasterize=True, clim=(-1., 1.))#, geo=True) #* gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
otp.impl.Dataset.S1.hvplot(x='XZ',y='YZ',width=800, height=600,cmap='viridis', geo=True, dynamic=True, rasterize=True, clim=(-1., 1.)) * gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
otp.impl.Dataset.S1.isel(time=10).hvplot.contour(x='XZ',y='YZ',width=800, height=600,cmap='viridis', levels = 30)

In [ ]:
otp.impl.Dataset.S1.isel(time=10).hvplot.contourf(x='XZ',y='YZ',width=800, height=600,cmap='viridis', levels = 30)

In [ ]:
otp.impl.Dataset.S1.hvplot.line(width=800, height=600)

In [ ]:
otp.impl.Dataset.S1.isel(time=10).hvplot.points(x='XZ',y='YZ', geo=True, datashade=True, width=800, height=600) * gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
v = otp.impl.frames(['S1'],title='SSH', nv =20 )#,vmin=-.5, vmax=.5)
v

In [ ]:
v.save('/Users/brey/Downloads/ssh.mp4', writer='ffmpeg')

In [ ]:
video('/Users/brey/Downloads/ssh.mp4','mp4') # optional show mp4 video

In [ ]:
w = otp.impl.frames(['U1Z','V1Z'],title='Vel')#,scale=.1)
w

In [ ]:
w.save('/Users/brey/Downloads/uv.mp4', writer='ffmpeg')

In [ ]:
video('/Users/brey/Downloads/uv.mp4','mp4') # optional show mp4 video

## Observation points / Validation

In [ ]:
otp.impl.obs.locations

In [ ]:
# Get with index number
p = otp.impl.obs.iloc(210)
print (p.head())
print (p.tail())

In [ ]:
#get with Station Name  
p = otp.impl.obs.loc('UK - Ullapool')
p.head()

In [ ]:
#plot
ax = p.plot()

### sample nearest point from simulation 

In [ ]:
plat,plon = otp.impl.obs.locations.loc[32,['lat','lon']]
plat, plon

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.subplot(projection=ccrs.PlateCarree());

gp = ax.contourf(otp.impl.dem.longitude, otp.impl.dem.latitude, otp.impl.dem.bathymetry, transform=ccrs.PlateCarree());

pp = ax.scatter(plon, plat, marker='x', s=50, color='r', label='obs locations', transform=ccrs.PlateCarree());



In [ ]:
#ts = otp.vars.S1.sel(XZ=[lon], method='nearest', tolerance=5).sel(YZ=[lat], method='nearest', tolerance=5)
#ts = ts.squeeze().to_pandas()
#ts.head()

In [ ]:
ts = point(lon=plon,lat=plat,data=otp.impl)
ts.tseries(var='S1')

In [ ]:
ts.S1.head()

In [ ]:
ts.S1.plot()

In [ ]:
## Join the graphs
ax = p.plot(figsize=(14,10))
ts.S1.plot(ax=ax,color=['r'])
ax.legend(list(p.columns.values) + ['D3D'], loc='center left',bbox_to_anchor=(1.1, 0.5))

...or...

In [ ]:
## Join the graphs
p.hvplot(figsize=(14,10)) * ts.S1.hvplot(label='d3d',color='red')

In [ ]:
otp.impl.Dataset.PATM.hvplot.image('XZ','YZ',groupby='time', width=500,colormap='jet', dynamic=False)